In [4]:
import pandas as pd
import json
from FoodicsAPIMethods import FoodAPIMethods

In [17]:
class Process_Transactions:
    def __init__(self, authURL):
        self.fm_api = FoodAPIMethods(authURL)
        
        
    def Convert_Json_INTO_DF(self, url, current_page = 1):
        print(url + '&' + 'page=' + str(current_page))
        endpointURL = url + '&' + 'page=' + str(current_page)
        data_json = self.fm_api.Get_Transaction(endpointURL)
        extracted_data = [{'id': transaction['id'], 'reference': transaction['reference']} for transaction in data_json['data']]
        data_frame = pd.DataFrame(extracted_data)
        if data_json['meta']['last_page'] != 1 and current_page < data_json['meta']['last_page']:
            next_page = current_page + 1
            new_df = self.Convert_Json_INTO_DF(url, current_page = next_page)
            data_frame = pd.concat([data_frame, new_df], ignore_index=True)
        return data_frame
        

In [18]:
PT = Process_Transactions('https://console-sandbox.foodics.com/dashboard?code=def5020082c06093752f02e896626304960e4ba87003b6b8b0123e753f5f70f4439c996c5de910d06f60b5bd5b346617429425b67c3a9b9f06af2702478d2c2cedf7b64278ccfdf3d571848e1071a83109b44f3d599fd8465b92b2c7a0307da00a452489756d9bfebe3e45e983d28463d588996b610074b66190d40b0f72769b22dee86e02e4809ba1c0354a24f5472fcad4436067474268f56dadd15471fdfc8d446c45179d328be88585fc92a696d7968e348950eb95f5da809fdca12f2d20917d0ad797c71c6bcad5ee38157ac741bfa511ffbce38b1011f9218848c6978f2bc34d56a3ed1195547d8dd655a39712fe661eeedfb7f75680be1bef7b18e1a3fc971ed3ee056634890b10ddd22b6acfe545ded668a2c980cde0a0994f88962e5b342303cd6b2eaeb2b411010bed8facb86d6d172eb289f1ce8b31c2f7db2fb1e080f8e47533f6042ec9ccc8c303097263d1595c3d0253b69d8da9ba7f62291e9f52a7784cfed932f31a9b227d33ae47f59a21f5e3fa95888bedbce281b0594341a9f4e82bec7fe4e8d82899784c8c318e01fa7906207923ff5eb283a41b9491792dea1c64142ececc8b197d1bf96beedda16cf28e&state=random_string')

In [19]:
# 'https://api-sandbox.foodics.com/v5/orders/9c2a63bd-ae00-4eca-b336-9aa16f15fbf8'
df = PT.Convert_Json_INTO_DF('https://api-sandbox.foodics.com/v5/orders?sort=reference')

https://api-sandbox.foodics.com/v5/orders?sort=reference&page=1
https://api-sandbox.foodics.com/v5/orders?sort=reference&page=2
https://api-sandbox.foodics.com/v5/orders?sort=reference&page=3
https://api-sandbox.foodics.com/v5/orders?sort=reference&page=4


In [20]:
# !jupyter nbconvert --to script FoodicsAPIMethods.ipynb

In [21]:
dublicate_ids = df['reference'].duplicated()
print(len(dublicate_ids))

176


In [22]:
df[dublicate_ids]

,id,reference


In [23]:
with pd.ExcelWriter('Result/f1.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='sheet', index=False)